## Stat Analysis on just R4 VS R72 DIFFs

In [1]:
import sys # note temporary... 
sys.path.append("..") # note temporary...

import re 
import pandas as pd
import numpy as np
from key_interactions_finder import pycontact_processing
from key_interactions_finder import data_preperation
from key_interactions_finder import stat_modelling
from key_interactions_finder import post_proccessing
from key_interactions_finder import pymol_projections

#### R4 + R7_2 load in and take only conf C frames from both datasets... 

In [2]:
# all input + output files 
R4_in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Datasets\R4_3iio"
R4_classifications_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Datasets\R4_3iio\R4_3iio_1in10_Classified.dat"

R7_2_in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Datasets\R7_2_5d38"
R7_2_classifications_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Datasets\R7_2_5d38\R7_2_5d38_1in10_Classified.dat"

out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              "PyContact_Per_Frame_Interactions_Block13.csv", "PyContact_Per_Frame_Interactions_Block14.csv",
                              "PyContact_Per_Frame_Interactions_Block15.csv", "PyContact_Per_Frame_Interactions_Block16.csv",
                              "PyContact_Per_Frame_Interactions_Block17.csv"]

# dataset prep parts. 
R4_pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=R4_in_dir,
)

# SPECIAL STEP FOR KEMP ELIMINASES BECAUSE PYCONTACT RENUMBERED THEM TO 0. 
R4_prepared_renumberd_df = pycontact_processing.modify_column_residue_numbers(
    dataset=R4_pycontact_dataset.prepared_df, 
    constant_to_add=1
)

R4_supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=R4_prepared_renumberd_df,
    target_file=R4_classifications_file,
    is_classification=True,
    header_present=True 
)

R4_supervised_dataset.df_processed.head(3)

Your PyContact file(s) have been succefully processed.
You have 1777 features and 10001 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


,Target,2Leu 247Asn Hbond bb-sc,3Ala 247Asn Hbond bb-bb,3Ala 248Val Other sc-bb,3Ala 249Arg Hbond bb-sc,4Lys 248Val Other bb-bb,4Lys 247Asn Hbond bb-bb,4Lys 219Asp Hbond sc-bb,4Lys 214Phe Hbond sc-sc,4Lys 246Val Other sc-sc,...,242Lys 245Gly Hbond bb-bb,242Lys 250Leu Hbond sc-sc,247Asn 1Ala Hbond sc-bb,242Lys 247Asn Hbond sc-sc,245Gly 1Ala Hbond bb-sc,246Val 1Ala Other bb-sc,242Lys 249Arg Hbond sc-bb,249Arg 1Ala Hbond sc-sc,250Leu 1Ala Hydrophobic sc-sc,244His 1Ala Other bb-sc
0,ConfA,6.32527,6.97320,2.43971,0.52622,0.80964,3.75249,4.67685,5.81246,0.46333,...,1.76301,1.01272,2.97377,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,ConfA,3.11127,12.17355,0.44933,0.02378,1.49925,4.05440,5.39872,3.17257,0.59219,...,3.06733,0.79475,1.25536,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,ConfA,0.01249,7.34447,1.09589,0.04325,0.26317,4.18954,2.84476,4.95749,0.67065,...,3.35582,0.13416,0.00000,0.05817,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# dataset prep parts. 
R7_2_pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=R7_2_in_dir,
)

# SPECIAL STEP FOR KEMP ELIMINASES BECAUSE PYCONTACT RENUMBERED THEM TO 0. 
R7_2_prepared_renumberd_df = pycontact_processing.modify_column_residue_numbers(
    dataset=R7_2_pycontact_dataset.prepared_df, 
    constant_to_add=1
)

R7_2_supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=R7_2_prepared_renumberd_df,
    target_file=R7_2_classifications_file,
    is_classification=True,
    header_present=True 
)

R7_2_supervised_dataset.df_processed.head(3)

Your PyContact file(s) have been succefully processed.
You have 1696 features and 10001 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


,Target,2Leu 247Asn Hbond bb-sc,3Ala 247Asn Other bb-bb,3Ala 248Val Other sc-bb,3Ala 249Arg Hbond bb-sc,3Ala 219Asp Other bb-bb,4Lys 219Asp Hbond sc-bb,4Lys 247Asn Hbond bb-bb,4Lys 249Arg Hbond bb-sc,4Lys 248Val Other bb-bb,...,242Lys 248Val Hbond sc-sc,247Asn 1Ala Hbond sc-bb,242Lys 250Leu Hbond sc-sc,245Gly 1Ala Other bb-sc,242Lys 247Asn Hbond sc-sc,246Val 1Ala Other bb-sc,242Lys 249Arg Hbond sc-bb,249Arg 1Ala Hbond sc-sc,246Val 250Leu Other bb-sc,248Val 1Ala Other bb-bb
0,None,3.73759,4.74069,0.47376,1.75461,0.03624,4.16713,3.26746,4.95453,1.46961,...,0.00742,2.59952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,None,1.38026,9.77922,1.96493,1.24201,0.00000,3.00073,5.33204,4.81332,4.47607,...,0.00729,3.10867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,None,0.29787,6.96840,3.22936,0.62743,0.07521,4.51820,3.76631,0.20769,1.52972,...,0.00000,1.87265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Now Merge both datasets.

First make deep copys of all frames with ConfC and then edit the ConfC name to be unique.
Then modify all mutated residues so that they will match when I merge the columns.. 

I will rename them all to have the R7_2 naming style so I can still make pymol projections... 

In [4]:
R4_confC_df = (R4_supervised_dataset.df_processed[(R4_supervised_dataset.df_processed["Target"] == "ConfC")]).copy(deep=True)
R7_2_confC_df = (R7_2_supervised_dataset.df_processed[(R7_2_supervised_dataset.df_processed["Target"] == "ConfC")]).copy(deep=True)
R7_2_confC_df.head(3)

,Target,2Leu 247Asn Hbond bb-sc,3Ala 247Asn Other bb-bb,3Ala 248Val Other sc-bb,3Ala 249Arg Hbond bb-sc,3Ala 219Asp Other bb-bb,4Lys 219Asp Hbond sc-bb,4Lys 247Asn Hbond bb-bb,4Lys 249Arg Hbond bb-sc,4Lys 248Val Other bb-bb,...,242Lys 248Val Hbond sc-sc,247Asn 1Ala Hbond sc-bb,242Lys 250Leu Hbond sc-sc,245Gly 1Ala Other bb-sc,242Lys 247Asn Hbond sc-sc,246Val 1Ala Other bb-sc,242Lys 249Arg Hbond sc-bb,249Arg 1Ala Hbond sc-sc,246Val 250Leu Other bb-sc,248Val 1Ala Other bb-bb
453,ConfC,1.41273,10.29339,1.42667,4.94727,0.00000,4.71024,4.01988,4.94206,0.05205,...,0.10859,1.42497,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0
493,ConfC,2.47360,9.23217,0.00000,0.00000,0.09594,5.34521,4.14228,2.80674,0.45835,...,0.82827,3.80277,0.74556,0.01732,0.0,0.00000,0.0,0.0,0.0,0.0
495,ConfC,2.46814,8.96298,0.03512,0.00000,0.03896,6.00282,4.79368,0.41854,1.21820,...,0.03247,3.69086,0.52728,0.51224,0.0,0.00969,0.0,0.0,0.0,0.0


In [5]:
R4_confC_df["Target"] = R4_confC_df["Target"].str.replace("ConfC", "R4_ConfC")
R7_2_confC_df["Target"] = R7_2_confC_df["Target"].str.replace("ConfC", "R7_2_ConfC")
R4_confC_df.head(3)

,Target,2Leu 247Asn Hbond bb-sc,3Ala 247Asn Hbond bb-bb,3Ala 248Val Other sc-bb,3Ala 249Arg Hbond bb-sc,4Lys 248Val Other bb-bb,4Lys 247Asn Hbond bb-bb,4Lys 219Asp Hbond sc-bb,4Lys 214Phe Hbond sc-sc,4Lys 246Val Other sc-sc,...,242Lys 245Gly Hbond bb-bb,242Lys 250Leu Hbond sc-sc,247Asn 1Ala Hbond sc-bb,242Lys 247Asn Hbond sc-sc,245Gly 1Ala Hbond bb-sc,246Val 1Ala Other bb-sc,242Lys 249Arg Hbond sc-bb,249Arg 1Ala Hbond sc-sc,250Leu 1Ala Hydrophobic sc-sc,244His 1Ala Other bb-sc
1084,R4_ConfC,3.04346,5.37607,1.38871,0.01718,0.14463,3.19382,4.42195,9.97912,1.35277,...,2.88999,0.06787,1.35923,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1119,R4_ConfC,1.93191,4.25898,1.46018,0.51111,0.37213,3.82111,4.67633,9.75317,1.16058,...,3.55481,0.24743,1.16506,0.62234,0.0,0.0,0.0,0.0,0.0,0.0
1120,R4_ConfC,3.57483,1.70229,0.00000,0.00000,1.62216,6.96758,5.79838,4.03679,0.91824,...,3.55208,1.11757,1.95837,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


#### mutuational diffs are: V12M, E146T, F77I, F229S, I102F, 

In [6]:
display([col for col in R4_confC_df.columns if "12Val" in col])
display([col for col in R4_confC_df.columns if "146Glu" in col])
display([col for col in R4_confC_df.columns if "77Phe" in col])
display([col for col in R4_confC_df.columns if "229Phe" in col])
display([col for col in R4_confC_df.columns if "102Ile" in col])

['12Val 18Val Other bb-sc',
 '12Val 16Arg Other sc-bb',
 '12Val 50Trp Hbond bb-bb',
 '12Val 51Asp Other bb-bb',
 '12Val 17Val Other sc-bb',
 '12Val 65Leu Hydrophobic sc-sc',
 '12Val 32Pro Hydrophobic sc-sc',
 '12Val 49Phe Hydrophobic sc-sc',
 '12Val 52Ile Hbond bb-bb',
 '12Val 53Thr Hbond bb-bb',
 '12Val 61Thr Other sc-sc',
 '12Val 62Met Hydrophobic sc-sc']

['130Ala 146Glu Other sc-bb',
 '132Lys 146Glu Saltbr sc-sc',
 '141Phe 146Glu Other sc-bb',
 '142Thr 146Glu Hbond bb-bb',
 '143Tyr 146Glu Hbond bb-sc',
 '146Glu 173Ile Other bb-sc',
 '146Glu 176Leu Other sc-sc',
 '146Glu 202Arg Saltbr sc-sc',
 '146Glu 175Arg Saltbr sc-sc']

['46Glu 77Phe Other sc-bb',
 '47Leu 77Phe Other bb-bb',
 '48Ser 77Phe Other bb-sc',
 '49Phe 77Phe Hydrophobic sc-sc',
 '66Val 77Phe Hydrophobic sc-sc',
 '69Val 77Phe Hydrophobic sc-sc',
 '70Ala 77Phe Other bb-sc',
 '73Ile 77Phe Hydrophobic sc-sc',
 '65Leu 77Phe Other bb-sc',
 '67Glu 77Phe Other bb-sc',
 '62Met 77Phe Hydrophobic sc-sc',
 '77Phe 98Asp Hbond bb-sc',
 '77Phe 97Ala Other sc-bb',
 '77Phe 96Gly Other sc-bb',
 '77Phe 99Lys Hbond bb-bb',
 '77Phe 95Arg Other sc-bb',
 '77Phe 222Lys Other bb-sc',
 '77Phe 100Val Other bb-bb']

['18Val 229Phe Other bb-sc',
 '19Lys 229Phe Other bb-sc',
 '20Gly 229Phe Other bb-sc',
 '21Ser 229Phe Other bb-sc',
 '22Asn 229Phe Hbond sc-bb',
 '23Phe 229Phe Other sc-bb',
 '24Glu 229Phe Other sc-sc',
 '179Lys 229Phe Other sc-sc',
 '225Ser 229Phe Hbond bb-sc',
 '226Val 229Phe Hbond bb-bb',
 '224Asp 229Phe Other bb-bb']

['50Trp 102Ile Other sc-bb',
 '83Ile 102Ile Other bb-sc',
 '84His 102Ile Other bb-sc',
 '89Ala 102Ile Hydrophobic sc-sc',
 '81Gly 102Ile Other bb-bb',
 '85Asp 102Ile Other bb-sc',
 '79Val 102Ile Other bb-bb',
 '82Gly 102Ile Other bb-bb',
 '102Ile 127Val Other bb-bb',
 '102Ile 128Tyr Hbond bb-bb',
 '102Ile 126Val Hbond bb-bb',
 '102Ile 112Leu Hydrophobic sc-sc',
 '102Ile 105Ala Hbond sc-bb',
 '102Ile 106Ala Hbond sc-sc',
 '102Ile 125Val Hydrophobic sc-sc',
 '102Ile 113Ile Hydrophobic sc-sc',
 '102Ile 116Ile Hydrophobic sc-sc',
 '102Ile 129Ile Other bb-bb',
 '102Ile 109Asn Other sc-bb',
 '102Ile 124Ala Other sc-bb']

In [7]:
# R4_confC_df.columns = R4_confC_df.columns.str.replace("12Val ", "V12M ")
# R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("12Met ", "V12M ")

# R4_confC_df.columns = R4_confC_df.columns.str.replace("146Glu ", "")
# R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("146Thr ", "")

# R4_confC_df.columns = R4_confC_df.columns.str.replace("77Phe ", "F77I ")
# R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("77Ile ", "F77I ")

# R4_confC_df.columns = R4_confC_df.columns.str.replace("229Phe ", "F229S ")
# R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("229Ser ", "F229S ")

# R4_confC_df.columns = R4_confC_df.columns.str.replace("102Ile ", "I102F ")
# R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("102Phe ", "I102F ")
# Does not work above for the feautre importance projection...

R4_confC_df.columns = R4_confC_df.columns.str.replace("12Val ", "12Met ")
R4_confC_df.columns = R4_confC_df.columns.str.replace("146Glu ", "146Thr ")
R4_confC_df.columns = R4_confC_df.columns.str.replace("77Phe ", "77Ile ")
R4_confC_df.columns = R4_confC_df.columns.str.replace("229Phe ", "229Ser ")
R4_confC_df.columns = R4_confC_df.columns.str.replace("102Ile ", "102Phe ")

In [8]:
display([col for col in R4_confC_df.columns if "12Val" in col])
display([col for col in R4_confC_df.columns if "146Glu" in col])
display([col for col in R4_confC_df.columns if "77Phe" in col])
display([col for col in R4_confC_df.columns if "229Phe" in col])
display([col for col in R4_confC_df.columns if "102Ile" in col])
# Now all empty, goood! 

[]

[]

[]

[]

[]

In [9]:
R4_confC_df.columns = R4_confC_df.columns.str.replace(" bb-sc", " bb-bb") 
R4_confC_df.columns = R4_confC_df.columns.str.replace(" sc-sc", " bb-bb") 
R4_confC_df.columns = R4_confC_df.columns.str.replace(" sc-bb", " bb-bb") 
R4_confC_df.columns

Index(['Target', '2Leu 247Asn Hbond bb-bb', '3Ala 247Asn Hbond bb-bb',
       '3Ala 248Val Other bb-bb', '3Ala 249Arg Hbond bb-bb',
       '4Lys 248Val Other bb-bb', '4Lys 247Asn Hbond bb-bb',
       '4Lys 219Asp Hbond bb-bb', '4Lys 214Phe Hbond bb-bb',
       '4Lys 246Val Other bb-bb',
       ...
       '242Lys 245Gly Hbond bb-bb', '242Lys 250Leu Hbond bb-bb',
       '247Asn 1Ala Hbond bb-bb', '242Lys 247Asn Hbond bb-bb',
       '245Gly 1Ala Hbond bb-bb', '246Val 1Ala Other bb-bb',
       '242Lys 249Arg Hbond bb-bb', '249Arg 1Ala Hbond bb-bb',
       '250Leu 1Ala Hydrophobic bb-bb', '244His 1Ala Other bb-bb'],
      dtype='object', length=1778)

In [10]:
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace(" bb-sc", " bb-bb") 
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace(" sc-sc", " bb-bb") 
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace(" sc-bb", " bb-bb") 
R7_2_confC_df.columns

Index(['Target', '2Leu 247Asn Hbond bb-bb', '3Ala 247Asn Other bb-bb',
       '3Ala 248Val Other bb-bb', '3Ala 249Arg Hbond bb-bb',
       '3Ala 219Asp Other bb-bb', '4Lys 219Asp Hbond bb-bb',
       '4Lys 247Asn Hbond bb-bb', '4Lys 249Arg Hbond bb-bb',
       '4Lys 248Val Other bb-bb',
       ...
       '242Lys 248Val Hbond bb-bb', '247Asn 1Ala Hbond bb-bb',
       '242Lys 250Leu Hbond bb-bb', '245Gly 1Ala Other bb-bb',
       '242Lys 247Asn Hbond bb-bb', '246Val 1Ala Other bb-bb',
       '242Lys 249Arg Hbond bb-bb', '249Arg 1Ala Hbond bb-bb',
       '246Val 250Leu Other bb-bb', '248Val 1Ala Other bb-bb'],
      dtype='object', length=1697)

### Now fixing issue that some identical interactions have different labelling on interaction types:
Pairs of interaction that need to be corrected, will edit the others to be correct.

In [11]:
matchme = "182Tyr "
display([col for col in R4_confC_df.columns if matchme in col])
display([col for col in R7_2_confC_df.columns if matchme in col])

['175Arg 182Tyr Hbond bb-bb',
 '178Thr 182Tyr Other bb-bb',
 '176Leu 182Tyr Other bb-bb',
 '177Gly 182Tyr Other bb-bb',
 '182Tyr 203Gly Hbond bb-bb',
 '182Tyr 212Glu Hbond bb-bb',
 '182Tyr 202Arg Other bb-bb',
 '182Tyr 213Ala Hydrophobic bb-bb',
 '182Tyr 187Ile Hydrophobic bb-bb',
 '182Tyr 200Ala Hbond bb-bb',
 '182Tyr 209His Hbond bb-bb',
 '182Tyr 201His Other bb-bb',
 '182Tyr 221Ala Hbond bb-bb',
 '182Tyr 204Ala Other bb-bb',
 '182Tyr 222Lys Hbond bb-bb',
 '182Tyr 210Phe Hydrophobic bb-bb',
 '182Tyr 223Ala Hydrophobic bb-bb',
 '182Tyr 205Gly Hbond bb-bb',
 '182Tyr 206Lys Other bb-bb',
 '182Tyr 224Asp Other bb-bb',
 '182Tyr 226Val Hydrophobic bb-bb',
 '182Tyr 227Phe Hydrophobic bb-bb',
 '182Tyr 214Phe Hydrophobic bb-bb']

['175Arg 182Tyr Hbond bb-bb',
 '178Thr 182Tyr Other bb-bb',
 '170Leu 182Tyr Other bb-bb',
 '182Tyr 203Gly Hbond bb-bb',
 '182Tyr 209His Other bb-bb',
 '182Tyr 204Ala Hydrophobic bb-bb',
 '182Tyr 202Arg Other bb-bb',
 '182Tyr 187Ile Hydrophobic bb-bb',
 '182Tyr 213Ala Hydrophobic bb-bb',
 '182Tyr 200Ala Hbond bb-bb',
 '182Tyr 201His Other bb-bb',
 '182Tyr 210Phe Hydrophobic bb-bb',
 '182Tyr 221Ala Hydrophobic bb-bb',
 '182Tyr 222Lys Hbond bb-bb',
 '182Tyr 212Glu Hbond bb-bb',
 '182Tyr 223Ala Hydrophobic bb-bb',
 '182Tyr 205Gly Other bb-bb',
 '182Tyr 226Val Hydrophobic bb-bb',
 '182Tyr 211Leu Other bb-bb',
 '182Tyr 227Phe Hydrophobic bb-bb',
 '182Tyr 220Ala Other bb-bb']

In [12]:
R4_confC_df.columns = R4_confC_df.columns.str.replace("10Leu 49Phe Other bb-bb", "10Leu 49Phe Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("5Arg 45Asp Other bb-bb", "5Arg 45Asp Saltbr bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("182Tyr 204Ala Other bb-bb", "182Tyr 204Ala Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("8Ala 44Ile Other bb-bb", "8Ala 44Ile Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("62Met 92Leu Other bb-bb", "62Met 92Leu Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("223Ala 227Phe Other bb-bb", "223Ala 227Phe Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("23Phe 228His Other bb-bb", "23Phe 228His Hbond bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("83Ile 102Phe Other bb-bb", "83Ile 102Phe Hydrophobic bb-bb")
R4_confC_df.columns = R4_confC_df.columns.str.replace("42Ile 234Val Other bb-bb", "42Ile 234Val Hbond bb-bb")


R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("51Asp 81Gly Other bb-bb", "51Asp 81Gly Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("182Tyr 209His Other bb-bb", "182Tyr 209His Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("3Ala 247Asn Other bb-bb", "3Ala 247Asn Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("172Ser 186Met Other bb-bb", "172Ser 186Met Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("201His 222Lys Other bb-bb", "201His 222Lys Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("117Ala 122Ser Other bb-bb", "117Ala 122Ser Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("5Arg 249Arg Other bb-bb", "5Arg 249Arg Hbond bb-bb")
R7_2_confC_df.columns = R7_2_confC_df.columns.str.replace("235Arg 249Arg Other bb-bb", "235Arg 249Arg Hbond bb-bb")

## Now need to merge both dataframes together and carry on as normal... 

In [13]:
R4_R72_df = (pd.concat([R4_confC_df, R7_2_confC_df], ignore_index=True,)).fillna(0)
R4_R72_df

,Target,2Leu 247Asn Hbond bb-bb,3Ala 247Asn Hbond bb-bb,3Ala 248Val Other bb-bb,3Ala 249Arg Hbond bb-bb,4Lys 248Val Other bb-bb,4Lys 247Asn Hbond bb-bb,4Lys 219Asp Hbond bb-bb,4Lys 214Phe Hbond bb-bb,4Lys 246Val Other bb-bb,...,203Gly 225Ser Hbond bb-bb,205Gly 225Ser Other bb-bb,200Ala 210Phe Other bb-bb,209His 232Ile Other bb-bb,207Met 227Phe Hydrophobic bb-bb,214Phe 221Ala Hydrophobic bb-bb,223Ala 228His Hbond bb-bb,245Gly 1Ala Other bb-bb,246Val 250Leu Other bb-bb,248Val 1Ala Other bb-bb
0,R4_ConfC,3.04346,5.37607,1.38871,0.01718,0.14463,3.19382,4.42195,9.97912,1.35277,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,R4_ConfC,1.93191,4.25898,1.46018,0.51111,0.37213,3.82111,4.67633,9.75317,1.16058,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,R4_ConfC,3.57483,1.70229,0.00000,0.00000,1.62216,6.96758,5.79838,4.03679,0.91824,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,R4_ConfC,3.27074,3.33114,0.00000,0.00000,1.49397,6.45726,8.14705,0.98338,0.85914,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,R4_ConfC,0.99572,3.17007,0.82260,0.71908,0.34294,5.13960,2.64873,9.72838,0.55836,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,R7_2_ConfC,0.00000,5.01360,1.86796,2.19589,0.85338,4.12751,3.17954,4.80792,0.79249,...,0.0,0.0,0.0,0.0,0.0,2.03862,0.0,0.0,0.0,0.0
12991,R7_2_ConfC,0.07867,8.60379,0.36535,0.58031,1.00033,2.64480,3.94500,8.07685,0.87542,...,0.0,0.0,0.0,0.0,0.0,3.05630,0.0,0.0,0.0,0.0
12992,R7_2_ConfC,0.20631,5.26808,0.32738,0.59982,0.20516,4.29526,6.78802,6.20227,0.32920,...,0.0,0.0,0.0,0.0,0.0,3.88400,0.0,0.0,0.0,0.0
12993,R7_2_ConfC,0.19901,5.82621,1.60310,1.93836,0.97618,4.54527,2.43326,8.08450,0.66220,...,0.0,0.0,0.0,0.0,0.0,2.21599,0.0,0.0,0.0,0.0


In [14]:
R4_R72_Combined_classifications_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Datasets\R7_2_5d38\R4_R72_Combined_Classifications.dat"
R4_R72_df.to_csv(R4_R72_Combined_classifications_file, columns=["Target"], index=False)


In [15]:
# Generate new instance of supervised dataset to do the filtering again... 
combined_supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=R4_R72_df.drop(["Target"], axis=1),
    target_file=R4_R72_Combined_classifications_file,
    is_classification=True,
    header_present=True 
)

combined_supervised_dataset.df_processed

Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


,Target,2Leu 247Asn Hbond bb-bb,3Ala 247Asn Hbond bb-bb,3Ala 248Val Other bb-bb,3Ala 249Arg Hbond bb-bb,4Lys 248Val Other bb-bb,4Lys 247Asn Hbond bb-bb,4Lys 219Asp Hbond bb-bb,4Lys 214Phe Hbond bb-bb,4Lys 246Val Other bb-bb,...,203Gly 225Ser Hbond bb-bb,205Gly 225Ser Other bb-bb,200Ala 210Phe Other bb-bb,209His 232Ile Other bb-bb,207Met 227Phe Hydrophobic bb-bb,214Phe 221Ala Hydrophobic bb-bb,223Ala 228His Hbond bb-bb,245Gly 1Ala Other bb-bb,246Val 250Leu Other bb-bb,248Val 1Ala Other bb-bb
0,R4_ConfC,3.04346,5.37607,1.38871,0.01718,0.14463,3.19382,4.42195,9.97912,1.35277,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,R4_ConfC,1.93191,4.25898,1.46018,0.51111,0.37213,3.82111,4.67633,9.75317,1.16058,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,R4_ConfC,3.57483,1.70229,0.00000,0.00000,1.62216,6.96758,5.79838,4.03679,0.91824,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,R4_ConfC,3.27074,3.33114,0.00000,0.00000,1.49397,6.45726,8.14705,0.98338,0.85914,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,R4_ConfC,0.99572,3.17007,0.82260,0.71908,0.34294,5.13960,2.64873,9.72838,0.55836,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,R7_2_ConfC,0.00000,5.01360,1.86796,2.19589,0.85338,4.12751,3.17954,4.80792,0.79249,...,0.0,0.0,0.0,0.0,0.0,2.03862,0.0,0.0,0.0,0.0
12991,R7_2_ConfC,0.07867,8.60379,0.36535,0.58031,1.00033,2.64480,3.94500,8.07685,0.87542,...,0.0,0.0,0.0,0.0,0.0,3.05630,0.0,0.0,0.0,0.0
12992,R7_2_ConfC,0.20631,5.26808,0.32738,0.59982,0.20516,4.29526,6.78802,6.20227,0.32920,...,0.0,0.0,0.0,0.0,0.0,3.88400,0.0,0.0,0.0,0.0
12993,R7_2_ConfC,0.19901,5.82621,1.60310,1.93836,0.97618,4.54527,2.43326,8.08450,0.66220,...,0.0,0.0,0.0,0.0,0.0,2.21599,0.0,0.0,0.0,0.0


In [16]:
# double check both match up
R4_R72_df.equals(combined_supervised_dataset.df_processed)

True

In [17]:
# Filtering 
combined_supervised_dataset.reset_filtering()
combined_supervised_dataset.filter_by_occupancy_by_class(min_occupancy=25)
combined_supervised_dataset.filter_by_avg_strength(average_strength_cut_off=1.0)

,Target,9Ala 48Ser Other bb-bb,9Ala 228His Hbond bb-bb,9Ala 224Asp Other bb-bb,9Ala 223Ala Hbond bb-bb,99Lys 167Glu Saltbr bb-bb,99Lys 126Val Other bb-bb,99Lys 124Ala Other bb-bb,99Lys 123Gln Hbond bb-bb,98Asp 124Ala Other bb-bb,...,103Asn 106Ala Hbond bb-bb,102Phe 128Tyr Hbond bb-bb,102Phe 127Val Other bb-bb,102Phe 126Val Hbond bb-bb,102Phe 112Leu Hydrophobic bb-bb,101Glu 128Tyr Other bb-bb,101Glu 126Val Other bb-bb,100Val 126Val Hbond bb-bb,100Val 125Val Other bb-bb,100Val 124Ala Hbond bb-bb
0,R4_ConfC,6.91506,3.91092,1.97500,0.12156,3.22338,1.76254,5.55861,1.50955,1.02750,...,4.10221,2.90030,3.36288,6.43720,1.35558,0.00000,3.51822,5.24184,0.83735,3.33914
1,R4_ConfC,3.35656,3.60941,2.22412,1.54951,3.43841,1.09623,3.69011,0.96439,1.79530,...,3.17674,1.93737,2.62044,5.04394,2.58339,0.02289,2.85628,5.86443,1.66706,5.27450
2,R4_ConfC,5.34275,3.02012,3.76926,0.39697,3.37118,1.44122,6.64202,2.54457,1.62666,...,4.59009,3.32417,3.56458,4.47816,2.21775,0.00000,3.19870,5.78156,1.57096,4.86908
3,R4_ConfC,3.38042,3.76975,1.92447,1.25602,3.45549,2.74933,3.27456,2.50071,1.90567,...,4.99628,2.42538,3.59649,3.72401,2.14941,0.00000,3.04834,5.83301,1.81236,5.60468
4,R4_ConfC,4.20082,3.11566,2.15577,1.21419,4.25493,2.04968,5.31345,2.22400,2.80219,...,3.51458,3.69531,3.70666,6.20736,2.57509,0.12428,2.17524,4.24681,0.79018,3.30311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,R7_2_ConfC,3.21681,1.85379,1.95897,3.70581,4.28523,2.40071,6.21249,2.74999,1.11818,...,3.93116,3.59392,2.51144,2.50178,6.58049,3.89344,4.71112,5.11118,1.81598,3.84729
12991,R7_2_ConfC,3.68385,2.49980,3.61342,3.72246,4.62577,0.82746,5.00681,1.16631,0.75125,...,3.71727,4.50169,4.41408,2.33053,4.66159,3.76382,5.17254,4.09752,2.02435,4.53825
12992,R7_2_ConfC,3.91976,2.26426,3.48892,3.94873,3.63166,1.20274,5.89317,1.20012,1.43391,...,4.31337,3.53842,3.38167,1.54050,7.62386,3.90581,3.90888,4.43838,2.96990,4.49998
12993,R7_2_ConfC,2.92675,2.07770,1.56319,4.13688,4.51257,1.39906,6.27495,1.62956,1.44905,...,4.11191,3.84800,3.66083,2.72510,3.45972,3.84301,4.60013,4.87308,2.10212,3.36269


In [18]:
combined_supervised_dataset.df_filtered["Target"].value_counts() 

R7_2_ConfC    7949
R4_ConfC      5046
Name: Target, dtype: int64

In [19]:
stat_model = stat_modelling.ClassificationStatModel(
    dataset=combined_supervised_dataset.df_filtered,
    class_names=["R4_ConfC", "R7_2_ConfC"],
    out_dir=out_dir,
    interaction_types_included=["Hbond", "Saltbr", "Hydrophobic", "Other"] 
)

In [20]:
stat_model.calc_js_distances()

Jensen-Shannon (JS) distances calculated.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/Jensen_Shannon_Per_Feature_Scores.csv written to disk.
You can also access these results via the class attribute: 'js_distances'.


In [21]:
stat_model.calc_mutual_info_to_target()

Mutual information scores calculated.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/Mutual_Information_Per_Feature_Scores.csv written to disk.
You can also access these results via the class attribute: 'mutual_infos'.


In [22]:
# First generate an instance of the class. 
post_proc = post_proccessing.StatClassificationPostProcessor(
    stat_model=stat_model,
    out_dir=out_dir
)

# Now we can run the get_per_res_importance() method, changing the stat_method accordingly.
js_per_res_importances = post_proc.get_per_res_importance(
    stat_method="jensen_shannon")
mi_per_res_importances = post_proc.get_per_res_importance(
    stat_method="mutual_information")

post_proc.estimate_feature_directions()

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/Jensen_Shannon_Distance_Scores_Per_Residue.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/Mutual_Information_Scores_Per_Residue.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/Feature_Direction_Estimates.csv written to disk.
You can access these predictions through the 'feature_directions' class attribute.


..\key_interactions_finder\post_proccessing.py:698: UserWarning: Warning, this method is very simplistic and just calculates the average contact score/strength for all features for both classes to determine the direction each feature appears to favour. You should therefore interpret these results with care...
  warnings.warn(warning_message)


In [23]:
# Write PyMOL compatable scripts for the per feature results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.js_distances,
    model_name="jensen_shannon",
    numb_features=100,
    out_dir=out_dir
)

pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.mutual_infos,
    model_name="mutual_information",
    numb_features=100,
    out_dir=out_dir
)

# Write PyMOL compatable scripts for the per residue results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_per_res_scores(
    per_res_scores=js_per_res_importances,
    model_name="jensen_shannon",
    out_dir=out_dir
)
pymol_projections.project_pymol_per_res_scores(
    per_res_scores=mi_per_res_importances,
    model_name="mutual_information",
    out_dir=out_dir
)

The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/jensen_shannon_Pymol_Per_Feature_Scores.py was written to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/mutual_information_Pymol_Per_Feature_Scores.py was written to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/jensen_shannon_Pymol_Per_Res_Scores.py was written to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72/mutual_information_Pymol_Per_Res_Scores.py was written to disk.


In [ ]:
combined_supervised_dataset.df_processed["50Trp 101Glu Hbond bb-bb"]

In [ ]:
R7_2_confC_df["50Trp 101Glu Hbond bb-bb"].mean()

In [ ]:
R4_confC_df["50Trp 101Glu Hbond bb-bb"].mean()

### Going to do Network Analysis on these two also

In [ ]:
from key_interactions_finder import network_analysis

In [ ]:
R4_confC_df = R4_confC_df.reset_index(drop=True)
R4_confC_df

In [ ]:
R4_confC_dataset = data_preperation.UnsupervisedFeatureData(
    input_df=R4_confC_df,
)

R4_confC_dataset.reset_filtering()
R4_confC_dataset.filter_by_occupancy(min_occupancy=50) # single class so same effect as by occupancy by_class method....
print(f"Number of features after filtering by average interaction scores: {len(R4_confC_dataset.df_filtered.columns)}")

In [ ]:
R7_2_confC_dataset = data_preperation.UnsupervisedFeatureData(
    input_df=R7_2_confC_df,
)

R7_2_confC_dataset.reset_filtering()
R7_2_confC_dataset.filter_by_occupancy(min_occupancy=50) # single class so same effect as by occupancy by_class method....
print(f"Number of features after filtering by average interaction scores: {len(R7_2_confC_dataset.df_filtered.columns)}")

In [ ]:
R4_pdb_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\1_PyMOL_Figures\Input_Structures\R4_3iio_XrayRef.pdb"
R4_correl_matrix_out_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Network\R4_3iio\ConfC_per_res_matrix.csv"
R4_distance_matrix_out_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Network\R4_3iio\ConfC_heavy_atom_contact_map.csv"


R4_correlation_network = network_analysis.CorrelationNetwork(
    dataset=R4_confC_dataset.df_filtered, 
)


per_res_corr_matrix = R4_correlation_network.gen_res_correl_matrix(
    out_file=R4_correl_matrix_out_file
)

heavy_atom_contact_map = network_analysis.heavy_atom_contact_map_from_pdb(
    pdb_file=R4_pdb_file,
    first_res=1,
    last_res=250,
    d_cut=5,
    out_file=R4_distance_matrix_out_file
)

In [ ]:
R7_2_pdb_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\1_PyMOL_Figures\Input_Structures\R7_2_5d38_XrayRef.pdb"
R7_2_correl_matrix_out_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Network\R7_2_5d38\ConfC_per_res_matrix.csv"
R7_2_distance_matrix_out_file=r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Network\R7_2_5d38\ConfC_heavy_atom_contact_map.csv"

R7_2_correlation_network = network_analysis.CorrelationNetwork(
    dataset=R7_2_confC_dataset.df_filtered, 
)


per_res_corr_matrix = R7_2_correlation_network.gen_res_correl_matrix(
    out_file=R7_2_correl_matrix_out_file
)

heavy_atom_contact_map = network_analysis.heavy_atom_contact_map_from_pdb(
    pdb_file=R7_2_pdb_file,
    first_res=1,
    last_res=250,
    d_cut=5,
    out_file=R7_2_distance_matrix_out_file
)

### Going to determine the closest distance to any mutation for each residue

In [ ]:
from key_interactions_finder import utils 

R4_out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\Dist_to_mutated_residues\R4_Dist_mutated_R4_R7_2_Mutates.txt"
R7_2_out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\Dist_to_mutated_residues\R7_2_Dist_mutated_R4_R7_2_Mutates.txt"
# V12M, E146T, F77I, F229S, I102F 

distances_to_site = utils.per_residue_distance_to_site(
    pdb_file=R4_pdb_file,
    out_file=R4_out_file,
    site_defintion="not name H* C CA N O and resid 12 146 77 229 102",
    first_residue=1,
    last_residue=250,
)

distances_to_site = utils.per_residue_distance_to_site(
    pdb_file=R7_2_pdb_file,
    out_file=R7_2_out_file,
    site_defintion="not name H* C CA N O and resid 12 146 77 229 102",
    first_residue=1,
    last_residue=250,
)

### Probability Distrubtions for the K222 key interactions 

In [24]:
x_values, selected_prob_distribs = post_proc.get_probability_distributions(
    number_features="all")

In [25]:
# ["R4_ConfC", "R7_2_ConfC"]
Lys222_keys = [key for key, value in selected_prob_distribs["R4_ConfC"].items() if '222Lys' in key]
Lys222_keys

['8Ala 222Lys Hbond bb-bb',
 '7Asp 222Lys Saltbr bb-bb',
 '48Ser 222Lys Hbond bb-bb',
 '201His 222Lys Hbond bb-bb',
 '200Ala 222Lys Hbond bb-bb',
 '199Ile 222Lys Other bb-bb',
 '182Tyr 222Lys Hbond bb-bb']

In [26]:
# '7Asp 222Lys Saltbr bb-bb'
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_D7_K222_SB_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['7Asp 222Lys Saltbr bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_D7_K222_SB_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['7Asp 222Lys Saltbr bb-bb'], delimiter=",")

# 8Ala 222Lys Hbond bb-bb
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_A8_K222_Hbond_bb_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['8Ala 222Lys Hbond bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_A8_K222_Hbond_bb_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['8Ala 222Lys Hbond bb-bb'], delimiter=",")

# '48Ser 222Lys Hbond bb-bb'
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_S48_K222_Hbond_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['48Ser 222Lys Hbond bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_S48_K222_Hbond_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['48Ser 222Lys Hbond bb-bb'], delimiter=",")


# '182Tyr 222Lys Hbond bb-bb'
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_Y182_K222_Hbond_sc_bb.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['182Tyr 222Lys Hbond bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_Y182_K222_Hbond_sc_bb.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['182Tyr 222Lys Hbond bb-bb'], delimiter=",")


# '200Ala 222Lys Hbond bb-bb'
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_A200_K222_Hbond_bb_bb.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['200Ala 222Lys Hbond bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_A200_K222_Hbond_bb_bb.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['200Ala 222Lys Hbond bb-bb'], delimiter=",")


# '201His 222Lys Hbond bb-bb'
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R4_ConfC_H201_K222_Hbond_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R4_ConfC"]['201His 222Lys Hbond bb-bb'], delimiter=",")

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\KE07_Workup\Raw_Results_Stats\ConfC_R4_vsR72\Prob_Distribs\R7_2_ConfC_H201_K222_Hbond_sc_sc.csv"
np.savetxt(out_file, selected_prob_distribs["R7_2_ConfC"]['201His 222Lys Hbond bb-bb'], delimiter=",")


In [27]:
# Occupancy - # '200Ala 222Lys Hbond bb-bb'

df = combined_supervised_dataset.df_filtered # 
df


,Target,9Ala 48Ser Other bb-bb,9Ala 228His Hbond bb-bb,9Ala 224Asp Other bb-bb,9Ala 223Ala Hbond bb-bb,99Lys 167Glu Saltbr bb-bb,99Lys 126Val Other bb-bb,99Lys 124Ala Other bb-bb,99Lys 123Gln Hbond bb-bb,98Asp 124Ala Other bb-bb,...,103Asn 106Ala Hbond bb-bb,102Phe 128Tyr Hbond bb-bb,102Phe 127Val Other bb-bb,102Phe 126Val Hbond bb-bb,102Phe 112Leu Hydrophobic bb-bb,101Glu 128Tyr Other bb-bb,101Glu 126Val Other bb-bb,100Val 126Val Hbond bb-bb,100Val 125Val Other bb-bb,100Val 124Ala Hbond bb-bb
0,R4_ConfC,6.91506,3.91092,1.97500,0.12156,3.22338,1.76254,5.55861,1.50955,1.02750,...,4.10221,2.90030,3.36288,6.43720,1.35558,0.00000,3.51822,5.24184,0.83735,3.33914
1,R4_ConfC,3.35656,3.60941,2.22412,1.54951,3.43841,1.09623,3.69011,0.96439,1.79530,...,3.17674,1.93737,2.62044,5.04394,2.58339,0.02289,2.85628,5.86443,1.66706,5.27450
2,R4_ConfC,5.34275,3.02012,3.76926,0.39697,3.37118,1.44122,6.64202,2.54457,1.62666,...,4.59009,3.32417,3.56458,4.47816,2.21775,0.00000,3.19870,5.78156,1.57096,4.86908
3,R4_ConfC,3.38042,3.76975,1.92447,1.25602,3.45549,2.74933,3.27456,2.50071,1.90567,...,4.99628,2.42538,3.59649,3.72401,2.14941,0.00000,3.04834,5.83301,1.81236,5.60468
4,R4_ConfC,4.20082,3.11566,2.15577,1.21419,4.25493,2.04968,5.31345,2.22400,2.80219,...,3.51458,3.69531,3.70666,6.20736,2.57509,0.12428,2.17524,4.24681,0.79018,3.30311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,R7_2_ConfC,3.21681,1.85379,1.95897,3.70581,4.28523,2.40071,6.21249,2.74999,1.11818,...,3.93116,3.59392,2.51144,2.50178,6.58049,3.89344,4.71112,5.11118,1.81598,3.84729
12991,R7_2_ConfC,3.68385,2.49980,3.61342,3.72246,4.62577,0.82746,5.00681,1.16631,0.75125,...,3.71727,4.50169,4.41408,2.33053,4.66159,3.76382,5.17254,4.09752,2.02435,4.53825
12992,R7_2_ConfC,3.91976,2.26426,3.48892,3.94873,3.63166,1.20274,5.89317,1.20012,1.43391,...,4.31337,3.53842,3.38167,1.54050,7.62386,3.90581,3.90888,4.43838,2.96990,4.49998
12993,R7_2_ConfC,2.92675,2.07770,1.56319,4.13688,4.51257,1.39906,6.27495,1.62956,1.44905,...,4.11191,3.84800,3.66083,2.72510,3.45972,3.84301,4.60013,4.87308,2.10212,3.36269


In [28]:
R7_2_ConfC	= df[df["Target"].str.contains("R7_2_ConfC")]
R4_ConfC = df[df["Target"].str.contains("R4_ConfC")]

In [29]:
def calc_occupancy(data_series):
    """ 
    Calc occupancy for a pandas series
    """ 
    tot_cols = len(data_series)
    occupied = data_series.astype(bool).sum(axis=0)

    return round((occupied / tot_cols ) * 100, 2)

In [30]:
# '7Asp 222Lys Saltbr bb-bb'
occupancy = calc_occupancy(R7_2_ConfC["7Asp 222Lys Saltbr bb-bb"])
print(f" Occpancy for R7_2_ConfC, 7Asp 222Lys Saltbr bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["7Asp 222Lys Saltbr bb-bb"])
print(f" Occpancy for R4_ConfC, 7Asp 222Lys Saltbr bb-bb  is: {occupancy}")
print(" ")

# 8Ala 222Lys Hbond bb-bb
occupancy = calc_occupancy(R7_2_ConfC["8Ala 222Lys Hbond bb-bb"])
print(f" Occpancy for R7_2_ConfC, 8Ala 222Lys Hbond bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["8Ala 222Lys Hbond bb-bb"])
print(f" Occpancy for R4_ConfC, 8Ala 222Lys Hbond bb-bb  is: {occupancy}")
print(" ")

# '48Ser 222Lys Hbond bb-bb'
occupancy = calc_occupancy(R7_2_ConfC["48Ser 222Lys Hbond bb-bb"])
print(f" Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["48Ser 222Lys Hbond bb-bb"])
print(f" Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: {occupancy}")
print(" ")

# '182Tyr 222Lys Hbond bb-bb'
occupancy = calc_occupancy(R7_2_ConfC["182Tyr 222Lys Hbond bb-bb"])
print(f" Occpancy for R7_2_ConfC, 182Tyr 222Lys Hbond bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["182Tyr 222Lys Hbond bb-bb"])
print(f" Occpancy for R4_ConfC, 182Tyr 222Lys Hbond bb-bb  is: {occupancy}")
print(" ")

# '200Ala 222Lys Hbond bb-bb'
occupancy = calc_occupancy(R7_2_ConfC["200Ala 222Lys Hbond bb-bb"])
print(f" Occpancy for R7_2_ConfC, 200Ala 222Lys Hbond bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["200Ala 222Lys Hbond bb-bb"])
print(f" Occpancy for R4_ConfC, 200Ala 222Lys Hbond bb-bb  is: {occupancy}")
print(" ")

# '201His 222Lys Hbond bb-bb'
occupancy = calc_occupancy(R7_2_ConfC["201His 222Lys Hbond bb-bb"])
print(f" Occpancy for R7_2_ConfC, 201His 222Lys Hbond bb-bb  is: {occupancy}")
occupancy = calc_occupancy(R4_ConfC["201His 222Lys Hbond bb-bb"])
print(f" Occpancy for R4_ConfC, 201His 222Lys Hbond bb-bb  is: {occupancy}")

 Occpancy for R7_2_ConfC, 7Asp 222Lys Saltbr bb-bb  is: 100.0
 Occpancy for R4_ConfC, 7Asp 222Lys Saltbr bb-bb  is: 99.43
 
 Occpancy for R7_2_ConfC, 8Ala 222Lys Hbond bb-bb  is: 93.68
 Occpancy for R4_ConfC, 8Ala 222Lys Hbond bb-bb  is: 89.42
 
 Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: 94.88
 Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: 98.63
 
 Occpancy for R7_2_ConfC, 182Tyr 222Lys Hbond bb-bb  is: 99.48
 Occpancy for R4_ConfC, 182Tyr 222Lys Hbond bb-bb  is: 56.58
 
 Occpancy for R7_2_ConfC, 200Ala 222Lys Hbond bb-bb  is: 100.0
 Occpancy for R4_ConfC, 200Ala 222Lys Hbond bb-bb  is: 85.26
 
 Occpancy for R7_2_ConfC, 201His 222Lys Hbond bb-bb  is: 100.0
 Occpancy for R4_ConfC, 201His 222Lys Hbond bb-bb  is: 60.68


In [31]:
# '7Asp 222Lys Saltbr bb-bb'
mean = round(R7_2_ConfC["7Asp 222Lys Saltbr bb-bb"].mean(), 2)
print(f" Mean for R7_2_ConfC, 7Asp 222Lys Saltbr bb-bb  is: {mean}")
mean = round(R4_ConfC["7Asp 222Lys Saltbr bb-bb"].mean(), 2)
print(f" Mean for R4_ConfC, 7Asp 222Lys Saltbr bb-bb  is: {mean}")
print(" ")

# 8Ala 222Lys Hbond bb-bb
mean = round(R7_2_ConfC["8Ala 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 8Ala 222Lys Hbond bb-bb  is: {mean}")
mean = round(R4_ConfC["8Ala 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 8Ala 222Lys Hbond bb-bb  is: {mean}")
print(" ")

# '48Ser 222Lys Hbond bb-bb'
mean = round(R7_2_ConfC["48Ser 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: {mean}")
meany = round(R4_ConfC["48Ser 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: {mean}")
print(" ")

# '182Tyr 222Lys Hbond bb-bb'
mean = round(R7_2_ConfC["182Tyr 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 182Tyr 222Lys Hbond bb-bb  is: {mean}")
mean = round(R4_ConfC["182Tyr 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 182Tyr 222Lys Hbond bb-bb  is: {mean}")
print(" ")

# '200Ala 222Lys Hbond bb-bb'
mean = round(R7_2_ConfC["200Ala 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 200Ala 222Lys Hbond bb-bb  is: {mean}")
mean = round(R4_ConfC["200Ala 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 200Ala 222Lys Hbond bb-bb  is: {mean}")
print(" ")

# '201His 222Lys Hbond bb-bb'
mean = round(R7_2_ConfC["201His 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 201His 222Lys Hbond bb-bb  is: {mean}")
mean = round(R4_ConfC["201His 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 201His 222Lys Hbond bb-bb  is: {mean}")

 Mean for R7_2_ConfC, 7Asp 222Lys Saltbr bb-bb  is: 6.95
 Mean for R4_ConfC, 7Asp 222Lys Saltbr bb-bb  is: 9.14
 
 Occpancy for R7_2_ConfC, 8Ala 222Lys Hbond bb-bb  is: 0.92
 Occpancy for R4_ConfC, 8Ala 222Lys Hbond bb-bb  is: 2.49
 
 Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: 2.82
 Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: 2.82
 
 Occpancy for R7_2_ConfC, 182Tyr 222Lys Hbond bb-bb  is: 2.35
 Occpancy for R4_ConfC, 182Tyr 222Lys Hbond bb-bb  is: 1.18
 
 Occpancy for R7_2_ConfC, 200Ala 222Lys Hbond bb-bb  is: 4.0
 Occpancy for R4_ConfC, 200Ala 222Lys Hbond bb-bb  is: 2.24
 
 Occpancy for R7_2_ConfC, 201His 222Lys Hbond bb-bb  is: 9.04
 Occpancy for R4_ConfC, 201His 222Lys Hbond bb-bb  is: 4.98


In [34]:
# '48Ser 222Lys Hbond bb-bb'
mean = round(R7_2_ConfC["48Ser 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: {mean}")
mean = round(R4_ConfC["48Ser 222Lys Hbond bb-bb"].mean(), 2)
print(f" Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: {mean}")

 Occpancy for R7_2_ConfC, 48Ser 222Lys Hbond bb-bb  is: 2.82
 Occpancy for R4_ConfC, 48Ser 222Lys Hbond bb-bb  is: 4.0
